In [1]:
import torch
import torch.nn as nn

/home/hl8967/.conda/envs/qtensor_ai/lib/python3.9/site-packages/tqdm-4.63.0-py3.9.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [71]:
def random_unitary_generator(n_batch, counts):
    module = nn.Linear(1,1)
    module.unitary = nn.Parameter(torch.rand(n_batch, counts, 2**2, 2**2, dtype=torch.cfloat))
    orthmod = nn.utils.parametrizations.orthogonal(module, name='unitary')
    results = orthmod.unitary.detach()
    #results = torch.cat([results[:,:,i].reshape(n_batch, counts, 1, 2, 2) for i in range(4)], axis=2)
    results = results.reshape(n_batch, 1, counts, 2,2,2,2)
    results = torch.permute(results, (0,1,2,4,3,6,5))
    return results

In [73]:
unitary = random_unitary_generator(1,1)[0,0]
unitary2 = unitary
permutation = [0] + [5 - i for i in range(1, 5)]
unitary2 = torch.permute(unitary2, tuple(permutation)).conj()
unitary = torch.permute(unitary, (0,2,1,4,3)).reshape(4, 4)
unitary2 = unitary2.reshape(4, 4)
torch.matmul(unitary, unitary2)

tensor([[ 1.0000e+00+3.8884e-09j,  7.4506e-08-5.9605e-08j,
          0.0000e+00+1.4901e-08j, -7.4506e-08+1.4901e-07j],
        [ 7.4506e-08+5.9605e-08j,  1.0000e+00+1.0263e-08j,
         -2.2352e-08-1.0431e-07j,  5.9605e-08-1.1921e-07j],
        [ 0.0000e+00-2.9802e-08j, -2.2352e-08+1.1921e-07j,
          1.0000e+00-1.0664e-08j, -8.9407e-08-4.4703e-08j],
        [-7.4506e-08-1.4901e-07j,  5.9605e-08+1.1921e-07j,
         -8.9407e-08+5.9605e-08j,  1.0000e+00+3.2949e-09j]])

In [29]:
permutation

[0, 4, 3, 2, 1]

In [60]:
a=torch.rand(4,4)
print(a)
b=a.reshape(2,2,2,2)
print(b)
b=torch.permute(b,(1,0,3,2))
print(torch.permute(b,(3,2,1,0)).reshape(4,4))

tensor([[0.9144, 0.7312, 0.2330, 0.0066],
        [0.0954, 0.9878, 0.6692, 0.9234],
        [0.9473, 0.3522, 0.3447, 0.6632],
        [0.7087, 0.2987, 0.3807, 0.3847]])
tensor([[[[0.9144, 0.7312],
          [0.2330, 0.0066]],

         [[0.0954, 0.9878],
          [0.6692, 0.9234]]],


        [[[0.9473, 0.3522],
          [0.3447, 0.6632]],

         [[0.7087, 0.2987],
          [0.3807, 0.3847]]]])
tensor([[0.9144, 0.0954, 0.9473, 0.7087],
        [0.7312, 0.9878, 0.3522, 0.2987],
        [0.2330, 0.6692, 0.3447, 0.3807],
        [0.0066, 0.9234, 0.6632, 0.3847]])
